## Extract planet data

In [1]:
# Import the necessary libraries
import geemap
import ee
import geopandas as gpd
import json
import pandas as pd

In [2]:
# Authentication and GEE initialization
ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [3]:
# Configurar el mapa
Map = geemap.Map(center=(12.20, -16.36), zoom=12)
Map

Map(center=[12.2, -16.36], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
Map.user_roi.getInfo()

In [5]:
# To ensure the creation of a geometry
geometry = ee.Geometry.Polygon([[[-16.453217, 12.240196],
   [-16.453217, 12.251142],
   [-16.427029, 12.251142],
   [-16.427029, 12.240196],
   [-16.453217, 12.240196]]])

Indexes to add:
EVI,
LSWI,
NDSI,
RNDVI,
NDRE,
MNDWI,
RVI,
NDMI,
NDVI, 
SR, 
GCVI, 
NDWI, 
VARI, 
GNDVI, 
GRVI, 
SAVI

In [6]:
# Load the shapefile
shapefile_path = '../DataIn/Parcelas_poligono.shp'
gdf = gpd.read_file(shapefile_path)

# Convert the GeoDataFrame to GeoJSON and then to a FeatureCollection
def geojson_to_ee(gdf):
    geojson = json.loads(gdf.to_json())
    features = [ee.Feature(ee.Geometry(feature['geometry']), feature['properties']) for feature in geojson['features']]
    return ee.FeatureCollection(features)

fc = geojson_to_ee(gdf)

# Add the FeatureCollection to the map
Map.addLayer(fc)

# Define the image collection and the area of interest
collection = ee.ImageCollection("projects/ee-jesusc461/assets/Imagenes_planet_folder/Collection_planet") \
              .filterDate('2022-07-01', '2024-01-30') \
              .filterBounds(geometry)

# Function to clip images to the geometry
def clip_image(image):
    return image.clip(geometry)

# Apply the clipping function to each image in the collection
collection = collection.map(clip_image)

# Function to add spectral indices
def addIndices(img):
    # Calculate existing indices
    NDVI = img.normalizedDifference(['nir', 'red']).rename('NDVI')
    SR = img.select('nir').divide(img.select('red')).rename('SR')
    GCVI = img.expression('(NIR / GREEN) - 1', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GCVI')
    NDWI = img.expression('(GREEN - NIR) / (GREEN + NIR)', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('NDWI')
    VARI = img.expression('(GREEN - RED) / (GREEN + RED - BLUE)', {'BLUE': img.select('blue'), 'RED': img.select('red'), 'GREEN': img.select('green')}).rename('VARI')
    GRVI = img.expression('NIR / GREEN', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GRVI')
    GNDVI = img.normalizedDifference(['nir', 'green']).rename('GNDVI')
    SAVI = img.expression('(NIR - RED) / ((NIR + RED + 0.5) * 1.5)', {'NIR': img.select('nir'), 'RED': img.select('red')}).rename('SAVI')
    # Additional indices using red edge
    NDRE = img.expression('(NIR - RED_EDGE) / (NIR + RED_EDGE)', {'NIR': img.select('nir'), 'RED_EDGE': img.select('rededge')}).rename('NDRE')
    NDVIre = img.expression('(NIR - RED_EDGE) / (NIR + RED_EDGE)', {'NIR': img.select('nir'), 'RED_EDGE': img.select('rededge')}).rename('NDVIre')
    REI = img.expression('(RED_EDGE - RED) / (RED_EDGE + RED)', {'RED_EDGE': img.select('rededge'), 'RED': img.select('red')}).rename('REI')
    
    # Add all calculated indices as bands to the image
    return img.addBands([
        NDVI, SR, GCVI, NDWI, VARI, GNDVI, GRVI, SAVI, 
        NDRE, NDVIre, REI
    ])

# Apply the function to the image collection
ps_1 = collection.map(addIndices)

# Create a composite image by averaging all images
composite = ps_1.mean().float()

# Mosaic images by date and average the tiles by date
def mosaic_by_date(date, collection):
    date_str = ee.Date(date).format('YYYY-MM-dd')
    date_collection = collection.filter(ee.Filter.date(ee.Date(date), ee.Date(date).advance(1, 'day')))
    mosaic = date_collection.mosaic().set('system:time_start', date).toFloat()  # Average and convert to float32
    return mosaic

# Get unique dates in the collection
dates = ee.List(ps_1.aggregate_array('system:time_start')).distinct()

# Create a new collection with mosaicked images averaged by date
mosaicked_collection = ee.ImageCollection(dates.map(lambda date: mosaic_by_date(date, ps_1)))

# Add layers to the map
Map.addLayer(composite, {'bands': ['nir', 'red', 'green'], 'min': 201, 'max': 2464}, 'RGB')
Map.addLayer(geometry)



In [7]:
# Convert GeoDataFrame to GeoJSON and then to FeatureCollection
def geojson_to_ee(gdf):
    geojson = json.loads(gdf.to_json())
    features = [ee.Feature(ee.Geometry(feature['geometry']), feature['properties']) for feature in geojson['features']]
    return ee.FeatureCollection(features)

fc = geojson_to_ee(gdf)


In [8]:

# Extract data for each polygon and each image
def extract_data(image):
    def zonal_stats(feature):
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=3,  #
            maxPixels=1e9
        )
        return feature.set(stats).set({'image_date': image.date().format('YYYY-MM-dd')})
    
    return fc.map(zonal_stats)

# Iterate on the collection and extract data
results = []
image_list = mosaicked_collection.toList(mosaicked_collection.size())

for i in range(mosaicked_collection.size().getInfo()):
    image = ee.Image(image_list.get(i))
    image = addIndices(image)
    data = extract_data(image).getInfo()
    
    for feature in data['features']:
        properties = feature['properties']
        properties['geometry'] = feature['geometry']
        results.append(properties)

In [9]:
# Convert the results to a DataFrame of pandas
df = pd.DataFrame(results)

# Export to CSV file
df.to_csv('../Results/output_indices_planet.csv', index=False)

print('Exportación completada.')

Exportación completada.
